In [40]:
import numpy as np
import pandas as pd
import csv
import xgboost
print(xgboost.__version__)
from matplotlib import pyplot
import xgboost as xgb
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import train_test_split


1.3.3


In [41]:
test_df = pd.read_csv('test.csv')
train_df = pd.read_csv('train.csv')

In [42]:
test_df.describe()
if train_df.isna().sum().sum() !=0:
    train_df = train_df.fillna(train_df.mean())

In [66]:
from sklearn.decomposition import PCA

N=3

y = train_df.loss
X = train_df.drop(columns=['loss'])

pca = PCA(n_components=N)
principalComponents = pca.fit_transform(X)
X = pd.DataFrame(data = principalComponents, columns = [('P' + str(i)) for i in range(N)])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)



(167500, 3)
(82500, 3)
(167500,)
(82500,)


In [67]:

model = xgb.XGBRegressor(n_estimators=800, max_depth=4, eta=0.1, subsample=0.7, colsample_bytree=0.8)

cv = RepeatedKFold(n_splits=4, n_repeats=3, random_state=1)

scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, verbose = 2)

scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )


ss0
ss1


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:  1.6min remaining:  3.3min


ss2
Mean MAE: 6.217 (0.023)


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  2.4min finished


In [68]:
yhat = model.predict(X_test)

NotFittedError: need to call fit or load_model beforehand